In [1]:
import pickle

def load_preprocessed_data(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

from process_data import save_image_pkl, save_depth_pkl, load_fish_depth
        

In [2]:
saved_folder_path = 'C:\\\\Users\\\\szxkd\\\\Desktop\\\\mmml\\\\camelmera\\\\models\\\\gym\\\\data'
save_image_pkl(saved_folder_path)

Processing: C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\T1
7
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [3]:
save_depth_pkl(saved_folder_path)

Processing: C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\T1
7
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [4]:
image_tensor = load_preprocessed_data("C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\\T1\\image_lcam_fish.pkl")
depth_tensor = load_preprocessed_data("C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\\T1\\depth_rcam_fish.pkl")

In [5]:
print(image_tensor[0].shape)
print(depth_tensor[0].shape)

AttributeError: 'list' object has no attribute 'shape'